# Balatro Poker Hand Notebook

Predict the probability of poker hands in a game of Balatro.

This notebook has miscellaneous sketches. The final script will be available in `balatro_stat.py`.

In [36]:
%load_ext autoreload
%autoreload 2

import os
import json
import math

from balatro_save_reader import BalatroSaveReader, BalatroCard, BalatroPokerHand
from poker_hands import find_poker_hands

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Balatro uses a save format that I haven't seen before. Possibly something about Lua or the LOVE game engine? It looks like: `{["key"]=value,}`. This section explores the file format and parses it into a Python object for easier reading.

In [ ]:
save_file = 'Balatro\\1\\save.jkr'
save_path = os.path.join(os.getenv('APPDATA'), save_file)

save = BalatroSaveReader(save_path)

# Print the raw file
# print(str(save.balatro_save_file))

# Print the parsed dictionary
# print(json.dumps(save.data, indent=4))

print(f'Deck has {len(save.deck())} cards.')

print('Poker Hands:')
print(*save.poker_hands())

print('Current Hand:')
print(*save.hand())

Combinatorics of brute forcing 'next hand' probabilities. How far can I brute force?

In [24]:
hand_size = 8
deck_size = 44

max_discard = 5

operation_count = 0
for discard_size in range(1, max_discard + 1):
    discard_combinations = math.comb(hand_size, discard_size)
    draw_combinations = math.comb(deck_size, discard_size)

    print(f'\t> Discarding {discard_size}: ({discard_combinations:_}) * ({draw_combinations:_})')
    operation_count += discard_combinations * draw_combinations

print()
print(f'Hand Size: {hand_size}; Deck Size: {deck_size}; Max Discards: {max_discard};')
print(f'Total Operations to Brute Force: {operation_count:_}')

	> Discarding 1: (8) * (44)
	> Discarding 2: (28) * (946)
	> Discarding 3: (56) * (13_244)
	> Discarding 4: (70) * (135_751)
	> Discarding 5: (56) * (1_086_008)

Hand Size: 8; Deck Size: 44; Max Discards: 5;
Total Operations to Brute Force: 71_087_522


In [43]:
def make_cards(cards) -> list[BalatroCard]:
    value_dict = { 'A': 'Ace', 'K': 'King', 'Q': 'Queen', 'J': 'Jack' }
    suit_dict = { 'C': 'Clubs', 'D': 'Diamonds', 'H': 'Hearts', 'S': 'Spades' }
    def parse(card):
        value, suit = card[:-1], card[-1:]
        value = value_dict[value.upper()] if value.upper() in value_dict else value
        suit = suit_dict[suit.upper()] if suit.upper() in suit_dict else suit
        return BalatroCard(value, suit)

    return [parse(card) for card in cards]

In [73]:
hands = [
    make_cards([ 'aS', 'qH', '10D', '8C', '6H', '4D', '3D', '2C']),
    make_cards([ 'aS', 'kH', 'qD', 'jC', '10C', '4H', '3D', '2C']),
    make_cards([ 'aC', 'kC', 'qC', 'jC', '10C']),
    make_cards([ 'aS', 'kH', 'kS', 'qC', 'qS', 'jC', 'jS', '10S', '2C']),
    make_cards([ 'aS', 'qH', '10D', '6C', '5C', '4H', '3D', '2C']),
    make_cards([ 'aS', 'qH', '7C', '6D', '5D', '4D', '3D', '2D']),
    make_cards([ '9H', '9S', '4D', '4H', '7S', '7D', '7C' ]),
    make_cards([ 'qH', '9H', '7S', '7H', '6H', '3H']),
]

for hand in hands:
    print(*hand, '->', find_poker_hands(hand))

A♠ Q♥ 10♦ 8♣ 6♥ 4♦ 3♦ 2♣ -> {'straight_flush': False, 'four_of_a_kind': False, 'full_house': False, 'flush': False, 'straight': False, 'two_pair': False, 'three_of_a_kind': False, 'pair': False}
A♠ K♥ Q♦ J♣ 10♣ 4♥ 3♦ 2♣ -> {'straight_flush': False, 'four_of_a_kind': False, 'full_house': False, 'flush': False, 'straight': True, 'two_pair': False, 'three_of_a_kind': False, 'pair': False}
A♣ K♣ Q♣ J♣ 10♣ -> {'straight_flush': True, 'four_of_a_kind': False, 'full_house': False, 'flush': True, 'straight': True, 'two_pair': False, 'three_of_a_kind': False, 'pair': False}
A♠ K♥ K♠ Q♣ Q♠ J♣ J♠ 10♠ 2♣ -> {'straight_flush': True, 'four_of_a_kind': False, 'full_house': False, 'flush': True, 'straight': True, 'two_pair': True, 'three_of_a_kind': False, 'pair': True}
A♠ Q♥ 10♦ 6♣ 5♣ 4♥ 3♦ 2♣ -> {'straight_flush': False, 'four_of_a_kind': False, 'full_house': False, 'flush': False, 'straight': True, 'two_pair': False, 'three_of_a_kind': False, 'pair': False}
A♠ Q♥ 7♣ 6♦ 5♦ 4♦ 3♦ 2♦ -> {'straight_flu